In [1]:
# Install pefile if not already installed
!pip install pefile

# Import required libraries
import pefile
import os
import re


  Obtaining dependency information for pefile from https://files.pythonhosted.org/packages/54/16/12b82f791c7f50ddec566873d5bdd245baa1491bac11d15ffb98aecc8f8b/pefile-2024.8.26-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/74.8 kB ? eta -:--:--
   ---------------- ----------------------- 30.7/74.8 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 74.8/74.8 kB 2.1 MB/s eta 0:00:00


In [2]:
suspicious_apis = [
    'GetProcAddress',
    'CreateRemoteThread',
    'TerminateProcess',
    'VirtualAlloc',
    'WriteProcessMemory'
]

def load_pe_file(file_path):
    try:
        pe = pefile.PE(file_path)
        print(f"✅ Successfully loaded {file_path}")
        return pe
    except pefile.PEFormatError:
        print(f"❌ Not a valid PE file: {file_path}")
        return None

def print_section_headers(pe):
    print("\nSection Headers:")
    for section in pe.sections:
        print(f"  - {section.Name.decode().strip()}")

def print_imported_libraries(pe):
    print("\nImported Libraries:")
    if hasattr(pe, 'DIRECTORY_ENTRY_IMPORT'):
        for entry in pe.DIRECTORY_ENTRY_IMPORT:
            print(f"  DLL: {entry.dll.decode()}")
            for imp in entry.imports:
                if imp.name:
                    print(f"    - {imp.name.decode()}")

def print_exported_symbols(pe):
    print("\nExported Symbols:")
    if hasattr(pe, 'DIRECTORY_ENTRY_EXPORT') and pe.DIRECTORY_ENTRY_EXPORT.symbols:
        for exp in pe.DIRECTORY_ENTRY_EXPORT.symbols:
            if exp.name:
                print(f"  - {exp.name.decode()}")
    else:
        print("  No exported symbols found.")

def extract_strings(file_path, min_length=4):
    print("\nStrings (length ≥ 4):")
    with open(file_path, 'rb') as f:
        data = f.read()
    strings = re.findall(rb"[ -~]{%d,}" % min_length, data)
    for s in strings[:30]:  # Show only 30 strings for readability
        print(f"  {s.decode(errors='ignore')}")

def check_suspicious_imports(pe):
    print("\nSuspicious Imports:")
    found = []
    if hasattr(pe, 'DIRECTORY_ENTRY_IMPORT'):
        for entry in pe.DIRECTORY_ENTRY_IMPORT:
            for imp in entry.imports:
                if imp.name:
                    name = imp.name.decode()
                    if name in suspicious_apis:
                        found.append(name)

    match_percent = (len(found) / len(suspicious_apis)) * 100
    print(f"  Matches: {found}")
    print(f"  Match %: {match_percent:.2f}%")

    if match_percent > 50:
        print("  ⚠️ MALICIOUS FILE DETECTED")
    else:
        print("  ✅ File is clean")


In [3]:
files = ["File1.exe", "File2.exe", "File3.exe"]

for file in files:
    print("="*70)
    print(f"📂 Analyzing: {file}")
    print("="*70)
    if os.path.exists(file):
        pe = load_pe_file(file)
        if pe:
            print_section_headers(pe)
            print_imported_libraries(pe)
            print_exported_symbols(pe)
            extract_strings(file)
            check_suspicious_imports(pe)
    else:
        print(f"❌ File not found: {file}")


📂 Analyzing: File1.exe
✅ Successfully loaded File1.exe

Section Headers:
  - .text   
  - .data   
  - .rsrc   

Imported Libraries:
  DLL: SHELL32.dll
    - ShellAboutW
  DLL: msvcrt.dll
    - __CxxFrameHandler
    - _CxxThrowException
    - wcstoul
    - toupper
    - wcschr
    - memmove
    - wcslen
    - _wcsrev
    - _c_exit
    - _exit
    - _XcptFilter
    - _cexit
    - exit
    - _acmdln
    - __getmainargs
    - _initterm
    - __setusermatherr
    - _adjust_fdiv
    - __p__commode
    - __p__fmode
    - __set_app_type
    - ??3@YAXPAX@Z
    - ??1type_info@@UAE@XZ
    - _controlfp
    - _except_handler3
    - ?terminate@@YAXXZ
  DLL: ADVAPI32.dll
    - RegOpenKeyExA
    - RegQueryValueExA
    - RegCloseKey
  DLL: KERNEL32.dll
    - GetModuleHandleA
    - LoadLibraryA
    - GetProcAddress
    - GlobalCompact
    - GlobalAlloc
    - GlobalFree
    - GlobalReAlloc
    - lstrcmpW
    - Sleep
    - WriteProfileStringW
    - GetStartupInfoA
    - GlobalSize
    - GlobalUnlock
    